In [ ]:
import json
import numpy as np
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from skimage.io import imread
from skimage.draw import polygon

# from IPython.display import display, Image, clear_output

import sys

sys.path.append("/src/") # path to the root directory
from visualization.visualize import print_image_with_point, show_sam_seg_img

In [ ]:
folder_path = "path/to/your/folder/"
image_shape = (1200, 1600)  # Replace with your image shape

# evaluation files
eval_path = folder_path + "data/cyprus_eval_frames.csv"
eval_path_results = folder_path + "data/cyprus_eval_frames_results.csv"
cyprus_eval_frames_results_predictions = folder_path + "data/cyprus_eval_frames_results_predictions.csv"

In [ ]:
path_mapping =  folder_path + "/label_mapping.json"
with open(path_mapping, 'r') as f:
    label_mapping = json.load(f)
    categories = label_mapping["categories"]
    categorie_mapping = label_mapping["category_mapping"]
categories

### definitions


In [ ]:
def labelme_json_to_mask_acc(json_file, image_shape):
    """
    Convert all LabelMe JSON annotations to one binary mask.

    :param json_file: Path to the LabelMe JSON file.
    :param image_shape: Shape of the corresponding image (height, width).
    :return: Binary mask as a numpy array.
    """
    with open(json_file) as f:
        data = json.load(f)
    
    mask = np.zeros(image_shape[:2], dtype=np.uint8)  # Assuming image_shape is (height, width, channels)
    
    # Iterate through all polygons (assuming 'polygon' type annotations)
    for shape in data['shapes']:
        if shape['shape_type'] == 'polygon':
            polygon_points = np.array(shape['points'])
            rr, cc = polygon(polygon_points[:, 1], polygon_points[:, 0], image_shape)
            mask[rr, cc] = 1
    return np.array(mask)


def labelme_json_to_mask(json_file, image_shape):
    """
    Convert LabelMe JSON annotations to a list of binary mask.

    :param json_file: Path to the LabelMe JSON file.
    :param image_shape: Shape of the corresponding image (height, width).
    :return:list of Binary mask as a numpy array.
    """
    with open(json_file) as f:
        data = json.load(f)
      
    masks = []
    # Iterate through all polygons (assuming 'polygon' type annotations)
    for shape in data['shapes']:
        mask = np.zeros(image_shape[:2], dtype=np.uint8)  # Assuming image_shape is (height, width, channels)
        if shape['shape_type'] == 'polygon':
            polygon_points = np.array(shape['points'])
            rr, cc = polygon(polygon_points[:, 1], polygon_points[:, 0], image_shape)
            mask[rr, cc] = 1
        np.array(mask)
        masks.append(mask)
    return masks

        
def labelme_json_to_mask_and_label(json_file, image_shape):
    """
    Convert LabelMe JSON annotations to a list of binary mask and labels.

    :param json_file: Path to the LabelMe JSON file.
    :param image_shape: Shape of the corresponding image (height, width).
    :return: list of Binary mask as a numpy array.
    :return: list of labels
    """
    with open(json_file) as f:
        data = json.load(f)
      
    masks = []
    labels = []
    # Iterate through all polygons (assuming 'polygon' type annotations)
    for shape in data['shapes']:
        mask = np.zeros(image_shape[:2], dtype=np.uint8)  # Assuming image_shape is (height, width, channels)
        if shape['shape_type'] == 'polygon':
            polygon_points = np.array(shape['points'])
            rr, cc = polygon(polygon_points[:, 1], polygon_points[:, 0], image_shape)
            mask[rr, cc] = 1
            masks.append(mask)
            labels.append(shape['label'])
    return masks, labels


def calculate_metrics_sam(ground_truth_mask, predicted_mask):
    """
    Calculate Precision, Recall, F1-Score, IoU, and Accuracy.

    :param ground_truth_mask: Binary mask of the ground truth.
    :param predicted_mask: Binary mask of the prediction.
    :return: Dictionary with metrics.
    """
    TP = np.sum((predicted_mask == 1) & (ground_truth_mask == 1))
    TN = np.sum((predicted_mask == 0) & (ground_truth_mask == 0))
    FP = np.sum((predicted_mask == 1) & (ground_truth_mask == 0))
    FN = np.sum((predicted_mask == 0) & (ground_truth_mask == 1))

    precision = TP / (TP + FP) if (TP + FP) else 0
    recall = TP / (TP + FN) if (TP + FN) else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    iou = TP / (TP + FP + FN) if (TP + FP + FN) else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) else 0

    return {
        'SAM_Precision': precision,
        'Sam_Recall': recall,
        'Sam_F1-Score': f1_score,
        'Sam_IoU': iou,
        'Sam_Accuracy': accuracy
    }
    
    
def calculate_metrics_seg(seg_mask, mask, label): # TODO just give the length 
    """
    input:
    - seg_mask: segmentation
    - masks: mask to check
    - label: correct label
    """
    
    total_px = seg_mask.shape[0] * seg_mask.shape[1]
    gt_pos_px = 0
    truepos_px = 0
    
    
    List_cat_count = np.zeros(len(categories))

    for x in range(0,seg_mask.shape[0]):
        for y in range(0,seg_mask.shape[1]):
            if mask[x,y] == True:
                gt_pos_px += 1
                id = seg_mask[int(x), int(y)]
                List_cat_count[id] += 1
                seg_label= categorie_mapping[categories[str(int(id))]]
                if seg_label == label:
                    truepos_px += 1
                    
    recall = truepos_px/gt_pos_px
                    
    return {
        'Seg_Total_px': total_px,
        'GT_Mask_size[px]': gt_pos_px,
        'True_prediction_in_mask': truepos_px,
        'Seg_recall': recall
    }, List_cat_count
    



In [ ]:
# load the names and print the categories
label_colors = {
        0: [255, 128, 0],    # Black for road
        1: [255, 0, 0],  # Red for label 1
        2: [0, 255, 0],  # Green for label 2
        3: [0, 0, 255],  # Blue for label 3
        4: [255, 255, 0],  # Yellow for label 4
        5: [255, 0, 255],  # Magenta for label 5
        6: [0, 255, 255],  # Cyan for label 6
        7: [128, 0, 0],    # Maroon for label 7
        8: [0, 128, 0],    # Green for label 8
        9: [0, 0, 128],    # Navy for label 9
        10: [128, 128, 0],  # Olive for label 10
        11: [128, 0, 128],  # Purple for label 11
        12: [0, 128, 128],  # Teal for label 12
        13: [192, 192, 192],  # Silver for label 13
        14: [128, 128, 128],  # Gray for label 14
        15: [0,0,0],  # Orange for label 15
        16: [0, 255, 128],  # Lime for label 16
        17: [128, 0, 255],  # Fuchsia for label 17
        18: [128, 255, 0],  # Lime for label 19
        # You can add more colors for additional labels as needed
    }

def get_seg_img(img,seg):
    """
    This function shows the image with the segmentation
    same ase show_img
    
    Parameters:
    - img: image
    - seg: segmentation
    
    Returns:
    - img: image with segmentation
    - color_seg: just segmentation 
    """
    # @TODO Rename to get_seg_img

    color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8) # height, width, 3\
    for label, color in label_colors.items():
        color_seg[seg == label] = color
    # Convert to BGR
    #color_seg = color_seg[..., ::-1] # kehrt die Reihenfolge der Farbkanäle um.

    # Show image + mask
    img = np.array(img) * 0.5 + color_seg * 0.5
    img = img.astype(np.uint8)

    return img, color_seg

def load_logits( seg_path):
    # Load logits from the specified path
    logits = torch.load(seg_path)   

    # Check if the shape matches the expected shape
    if logits.shape == torch.Size([1, 19, 128, 128]):
        logits = logits.float()  # Convert to 'float' data type
        logits = F.softmax(logits, dim=1)
        
        # Upsample the logits to match the image shape
        upsampled_logits = logits #nn.functional.interpolate(logits, size=img_shape, mode='bilinear', align_corners=False)
        # Get both the maximum probabilities and their corresponding labels
        max_probs, labels = torch.max(upsampled_logits, dim=1)
        label = labels[0]  # Extract the most probable label matrix
        prob = max_probs[0]  # Extract the probability matrix corresponding to the most probable labels
    
        return logits,label, prob  # Return both the label matrix and the probability matrix

### Calculate the Metrices for Semantic Segmentation and Sam

In [ ]:
df = pd.read_csv(eval_path_results)
data = df

total_acc = []
total_iou = []
total_f1 = []
total_recall = []
total_precision = []
recall_dict = []
precision_dict = []
seg_results = []
sam_results = []
results = []

# for all data in the evaluation set set
for i in range(0, len(data)):
    try:
        # load data
        x = data["x"].iloc[i]
        y = data["y"].iloc[i]
        session = data["session"].iloc[i]
        response = data["response"].iloc[i]
        frame_nr = data["frame_nr"].iloc[i]
        str_frame_nr = f"{frame_nr:05d}"

        # get image path
        path = folder_path + "/data/evalset/"
        handLabel_path = path + "handlabels/" + session + "_" + str_frame_nr + ".json"
        sam_mask_path = path + "sams/" + session + "_" + str_frame_nr + ".npy"
        seg_path = path + "segs/" + session + "_" + str_frame_nr + ".pt"
        image_path = path + "images/" + session + "_" + str_frame_nr + ".jpg"

        # load image
        image = imread(image_path)

        # load sam mask
        sam_mask = np.load(sam_mask_path)[0]
        labelme_mask, labels = labelme_json_to_mask_and_label(
            handLabel_path, image_shape
        )

        # load segmask
        seg_mask = load_logits(seg_path)[0]
        seg_mask = nn.functional.interpolate(
            seg_mask.float(), size=(1200, 1600), mode="bilinear", align_corners=False
        )
        seg_mask = torch.argmax(seg_mask, dim=1).squeeze(0).squeeze(0).numpy()

        # calculate metrics for each grount truth mask 
        #print(len(labelme_mask))
        for i, (mask, label) in enumerate(zip(labelme_mask, labels)):
            
            print(i)
            
            # calculate metrics sam
            metrics_sam = calculate_metrics_sam(mask, sam_mask)
            metrics_sam["session"] = session
            metrics_sam["frame_nr"] = frame_nr
            metrics_sam["mask_nr"] = i + 1
            sam_results.append(metrics_sam)

            # if the mask is labeld, calculte the metrics for the segmentation as well
            if label in set(categorie_mapping.values()):
                
                # calculate metrics seg
                metrics_seg, list_cat_count = calculate_metrics_seg(seg_mask, mask, label)

                # add the frame name to the dictionary
                metrics_seg["session"] = session
                metrics_seg["frame_nr"] = frame_nr
                metrics_seg["label"] = label
                metrics_seg["mask_nr"] = i + 1
                seg_results.append(metrics_seg)

            
            #### Start of the sample print -------------------------------------------------------
            PrintData = False
            if PrintData:#label == "ambiguous":  # metrics_seg["recall"] < 0.5:
                print(metrics_seg)
                for i, z in zip(list_cat_count, list(categories.values())):
                    if i > 0:
                        print(i, z)
                seg_s, _ = get_seg_img(image, seg_mask)

                color = [255, 0, 0]
                color_gr = np.zeros(
                    (1200, 1600, 3), dtype=np.uint8
                )  # height, width, 3\
                color_gr[mask == True] = color

                img_s = seg_s.copy()
                for x in range(0, seg_mask.shape[0]):
                    for y in range(0, seg_mask.shape[1]):
                        if mask[x, y] != True:
                            original_color = img_s[
                                x, y
                            ]  # Assuming img[x, y] is already in the form of a numpy array or similar
                            blend_color = [255, 255, 255]
                            blended_color = [
                                (int(original_color[i]) + blend_color[i]) // 2
                                for i in range(3)
                            ]
                            img_s[x, y] = blended_color

                img_mask = image.copy()
                for x in range(0, seg_mask.shape[0]):
                    for y in range(0, seg_mask.shape[1]):
                        if mask[x, y] != True:
                            original_color = img_mask[
                                x, y
                            ]  # Assuming img[x, y] is already in the form of a numpy array or similar
                            blend_color = [255, 255, 255]
                            blended_color = [
                                (int(original_color[i]) + 5 * blend_color[i]) // 6
                                for i in range(3)
                            ]
                            img_mask[x, y] = blended_color

                # plot image
                fig, axs = plt.subplots(1, 2, figsize=(15, 5))
                axs[0].imshow(image)
                axs[0].set_title("Original Image")
                axs[1].imshow(img_s)
                axs[1].set_title("Segmentation")
                # axs[2].imshow(mask, cmap='gray')
                # axs[2].set_title("Groundtruth Mask")
                for ax in axs:
                    ax.axis("off")
                plt.show()

                # plot one image with the segmentation
                plt.imshow(img_s)
                # axis of
                plt.axis('off')
                plt.show()

                # combine both masks
                color = [0, 255, 255]
                color_sam = np.zeros((1200, 1600, 3), dtype=np.uint8) # height, width, 3\
                color_sam[sam_mask == True] = color

                color = [255, 0, 0]
                color_gr = np.zeros((1200, 1600, 3), dtype=np.uint8) # height, width, 3\
                color_gr[mask == True] = color

                #img = color_sam * 0.5 + color_gr * 0.5
                img = seg_mask * 0.0 - mask * 0.8
                img = img.astype(np.uint8)

                # plot the mask onto the segmentation

                fig, axs = plt.subplots(1, 4, figsize=(20, 5))
                axs[0].imshow(image)
                axs[0].set_title("Original Image")
                axs[1].imshow(mask, cmap='gray')
                axs[1].set_title("Groundtruth Mask")
                axs[2].imshow(img)
                axs[2].set_title("Sam Mask")
               # axs[3].imshow(seg_mask)
                for ax in axs:
                    ax.axis('off')
                plt.show()

                     # combine both masks
                color = [0, 255, 255]
                color_sam = np.zeros((1200, 1600, 3), dtype=np.uint8) # height, width, 3\
                color_sam[sam_mask == True] = color

                color = [255, 0, 0]
                color_gr = np.zeros((1200, 1600, 3), dtype=np.uint8) # height, width, 3\
                color_gr[mask == True] = color

                #img = color_sam * 0.5 + color_gr * 0.5
                img = seg_mask * 0.0 - sam_mask * 0.8
                img = img.astype(np.uint8)

                # plot the mask onto the segmentation

                fig, axs = plt.subplots(1, 4, figsize=(20, 5))
                axs[0].imshow(image)
                axs[0].set_title("Original Image")
                axs[1].imshow(mask, cmap='gray')
                axs[1].set_title("Groundtruth Mask")
                axs[2].imshow(img)
                axs[2].set_title("Sam Mask")
               # axs[3].imshow(seg_mask)
                for ax in axs:
                    ax.axis('off')
                plt.show()
            #### End of the sample print -------------------------------------------------------

            # for each image just take one value
            # print("Max: Precision", max(result['Precision'] for result in all_metrics))
            # print("Recall", max(result['Recall'] for result in all_metrics))
            # print("F1-Score", max(result['F1-Score'] for result in all_metrics))
            # print("IoU", max(result['IoU'] for result in all_metrics))
            # print("Accuracy", max(result['Accuracy'] for result in all_metrics))
            # total_acc.append(max(result['Accuracy'] for result in all_metrics))
            # total_iou.append(max(result['IoU'] for result in all_metrics))
            # total_f1.append(max(result['F1-Score'] for result in all_metrics))
            # total_recall.append(max(result['Recall'] for result in all_metrics))
            # total_precision.append(max(result['Precision'] for result in all_metrics))
            # take the dictionary where the precision is the highes

            # metrics_sam
            # max_precision_dict = max(
            #     sam_results, key=lambda x: x["Precision"]
            # )
            # max_recall_dict = max(sam_results, key=lambda x: x["Recall"])
            # recall_dict.append(max_recall_dict)
            # precision_dict.append(max_precision_dict)

    except Exception as e:
        print(e)

In [ ]:
# store the results in a file
df_sam = pd.DataFrame(sam_results)
df_seg = pd.DataFrame(seg_results)

# merge such that no information is deltetd
df = pd.merge(df_sam, df_seg, on=["session", "frame_nr", "mask_nr"], how="outer")
df = df[['session', 'frame_nr','GT_Mask_size[px]', 'mask_nr','SAM_Precision', 'Sam_Recall', 'Sam_F1-Score', 'Sam_IoU',
    'Sam_Accuracy',  'Seg_Total_px',
    'True_prediction_in_mask', 'Seg_recall', 'label']]

# Save the DataFrame to a CSV file
df.to_csv(folder_path + '/data/eval_results_seg_sam.csv', index=False)